In [1]:
import ccxt
import os
import json
from datetime import datetime
from dotenv import load_dotenv

os.environ['TZ'] = 'UTC'
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [ ]:

def format_filename_date(timestamp):
    date = datetime.utcfromtimestamp(timestamp / 1000)
    return date.strftime('%Y-%m-%d')

In [ ]:
def fetch_all_ohlcv_data(exchange_id, symbol, start_date, end_date, timeframe):
    try:
        api_key = os.getenv(f'{exchange_id.upper()}_API_KEY')
        secret = os.getenv(f'{exchange_id.upper()}_API_SECRET')
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({'apiKey': api_key, 'secret': secret}) if api_key and secret else exchange_class()

        if exchange.has['fetchOHLCV']:
            all_ohlcv_data = []
            # Append "T00:00:00Z" to start and end dates to ensure correct parsing
            since = exchange.parse8601(f"{start_date}T00:00:00Z")
            until = exchange.parse8601(f"{end_date}T23:59:59Z")
            limit = 500  # or exchange's maximum limit if known

            while since < until:
                ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
                if ohlcv_data:
                    since = ohlcv_data[-1][0] + 1
                    all_ohlcv_data.extend(ohlcv_data)
                    print(f'Fetched data points up to {format_filename_date(ohlcv_data[-1][0])}')
                else:
                    break

            filtered_ohlcv_data = [
                {'Time': d[0], 'Open': d[1], 'High': d[2], 'Low': d[3], 'Close': d[4], 'Volume': d[5]}
                for d in all_ohlcv_data if d[0] <= until
            ]

            dir_path = './data'
            os.makedirs(dir_path, exist_ok=True)
            file_name = f"{dir_path}/{symbol.replace('/', '')}_{timeframe}.json"
            with open(file_name, 'w') as file:
                json.dump(filtered_ohlcv_data, file, indent=2)

            print(f'Data saved to {file_name}')
            return filtered_ohlcv_data
        else:
            print(f'Exchange {exchange_id} does not support fetchOHLCV.')
            return None
    except Exception as error:
        print(f'Error fetching data from {exchange_id}: {error}')

In [ ]:
# Call the function directly
fetch_all_ohlcv_data(
    "binance",
    "SOL/USDT",
    "2022-01-01",
    "2023-12-31",
    "1m"
)